In [1]:
import pandas as pd
import nltk
import multiprocessing as mp
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from nltk.corpus import stopwords
from tqdm import tqdm
from heapq import nlargest

import sys
# User defined Imports ugly python import syntax >:(
sys.path.append('./Preprocess')
from dataJoin import joinData
from parallelLoad import parallelLoad
from preprocess import CustomAnalyzer, doFreq

# Join Data

In [2]:
tqdm.pandas()
print('Loading data...')

# Start Data loading using paralelization parallelLoad(route_to_files) function!
filesRoute = './data/traditionalSpamBotsChunks1/'
botData = parallelLoad(filesRoute)
filesRoute = './data/genuineTweetsChunks/'
genuineData = parallelLoad(filesRoute)

print('Joining data...')
df = joinData(botData.head(20000), genuineData.head(20000))

# See how many tweets we read
print("Read {0:d} tweets".format(len(df)))

Loading data...
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk7.csv
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk2.csv
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk12.csv
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk4.csv
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk6.csv
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk14.csv
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk9.csv
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk15.csv
Starting the pooling...
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk8.csv
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk11.csv
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk13.csv
Loading the file:  ./data/traditionalSpamBotsChunks1/tweetsBots_chunk10.csv
Loading the file:  ./data/traditionalSpamBotsChunks1/t

Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk39.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk97.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk182.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk17.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk3.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk219.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk183.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk238.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk104.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk41.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk123.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk18.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk22.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine

Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk258.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk155.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk129.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk72.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk273.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk23.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk229.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk168.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk44.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk184.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk128.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk272.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGenuine_chunk218.csv
Loading the file:  ./data/genuineTweetsChunks/tweetsGe

./Preprocess/dataJoin.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  botData['bot'] = 1
./Preprocess/dataJoin.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  genuineData['bot'] = 0


In [3]:
import matplotlib.pyplot as plt 
plt.rc("font", size=14)

In [4]:
print(df.shape)
print(list(df.columns))

(40000, 26)
['id', 'text', 'source', 'user_id', 'truncated', 'in_reply_to_status_id', 'in_reply_to_user_id', 'in_reply_to_screen_name', 'retweeted_status_id', 'geo', 'place', 'contributors', 'retweet_count', 'reply_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'num_hashtags', 'num_urls', 'num_mentions', 'created_at', 'timestamp', 'crawled_at', 'updated', 'bot']


In [4]:
df.head()

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,...,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated,bot
0,21352627412,"Winning isnt everything, its the only thing. R...","<a href=""http://www.socialoomph.com"" rel=""nofo...",69744581.0,NaN,0.0,0.0,NaN,0.0,NaN,...,NaN,NaN,0.0,0.0,0.0,1281999866000L,2010-08-17 01:04:26,0000-00-00 00:00:00,2014-04-16 23:58:03,1
1,21352276280,"http://bit.ly/pro2020 This is a NO Brainer, tr...","<a href=""http://www.socialoomph.com"" rel=""nofo...",69744581.0,NaN,0.0,0.0,NaN,0.0,NaN,...,NaN,NaN,0.0,1.0,0.0,1281999559000L,2010-08-17 00:59:19,0000-00-00 00:00:00,2014-04-16 23:58:03,1
2,21351076660,"Winning is not everything, but the effort to w...","<a href=""http://www.socialoomph.com"" rel=""nofo...",69744581.0,NaN,0.0,0.0,NaN,0.0,NaN,...,NaN,NaN,0.0,0.0,0.0,1281998482000L,2010-08-17 00:41:22,0000-00-00 00:00:00,2014-04-16 23:58:03,1
3,21350081764,"One of my top mentors, so glad I joined him, s...","<a href=""http://www.socialoomph.com"" rel=""nofo...",69744581.0,NaN,0.0,0.0,NaN,0.0,NaN,...,NaN,NaN,0.0,1.0,0.0,1281997577000L,2010-08-17 00:26:17,0000-00-00 00:00:00,2014-04-16 23:58:03,1
4,21349577659,Winning is everything. The only ones who remem...,"<a href=""http://www.socialoomph.com"" rel=""nofo...",69744581.0,NaN,0.0,0.0,NaN,0.0,NaN,...,NaN,NaN,0.0,0.0,0.0,1281997111000L,2010-08-17 00:18:31,0000-00-00 00:00:00,2014-04-16 23:58:03,1


# Create dummy variables

In [5]:
cat_vars=['place','num_hashtags','num_urls','num_mentions']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df[var], prefix=var,prefix_sep='_',dummy_na=True)
    data1=df.join(cat_list)
    df=data1

cat_vars=['place','num_hashtags','num_urls','num_mentions']
data_vars=df.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

In [6]:
data_final=df[to_keep]
data_final.columns.values

array(['id', 'text', 'source', 'user_id', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_user_id',
       'in_reply_to_screen_name', 'retweeted_status_id', 'geo',
       'contributors', 'retweet_count', 'reply_count', 'favorite_count',
       'favorited', 'retweeted', 'possibly_sensitive', 'created_at',
       'timestamp', 'crawled_at', 'updated', 'bot', 'place_Alabama, USA',
       'place_Central Visayas, Republic of the Philippines',
       'place_Hudson, FL', 'place_Lungsod ng Butuan, Caraga',
       'place_Lungsod ng Cebu, Central Visayas',
       'place_Lungsod ng Mandaue, Central Visayas', 'place_Nashville, TN',
       'place_National Capital Region, Republic of the Philippines',
       'place_Ottawa, Ontario', 'place_Rutherford, TN',
       'place_Tennessee, USA', 'place_Verona, MS',
       'place_Western Visayas, Republic of the Philippines', 'place_nan',
       'num_hashtags_0.0', 'num_hashtags_1.0', 'num_hashtags_2.0',
       'num_hashtags_3.0', 'num_hashtags_4.0',

# Feature Selection

In [12]:
df = data_final
cols_to_remove = []

for col in df.columns:
    try:
        _ = df[col].astype(float)
    except ValueError:
        print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
data = df[[col for col in df.columns if col not in cols_to_remove]]
data = data.drop(['bot','id','user_id'], axis=1)
data = data.dropna(axis=1)
data.head()


Couldn't covert text to float
Couldn't covert source to float
Couldn't covert in_reply_to_screen_name to float
Couldn't covert created_at to float
Couldn't covert timestamp to float
Couldn't covert crawled_at to float
Couldn't covert updated to float


,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweet_count,reply_count,favorite_count,"place_Alabama, USA","place_Central Visayas, Republic of the Philippines","place_Hudson, FL","place_Lungsod ng Butuan, Caraga",...,num_mentions_3.0,num_mentions_4.0,num_mentions_5.0,num_mentions_6.0,num_mentions_7.0,num_mentions_8.0,num_mentions_9.0,num_mentions_10.0,num_mentions_11.0,num_mentions_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Recursive Feature Elimination

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 3)
rfe = rfe.fit(data, df.bot)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

/home/yifei/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yifei/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yifei/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yifei/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yifei/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 

/home/yifei/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yifei/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yifei/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yifei/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/yifei/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 

[ True False  True False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False]
[ 1  2  1  3  1 11 44 27 35 33 31 42 40 32 39 41 28 29 34 36 37  4  6  9
 14 24 26 30 23 43 46  8  7 12 38 45  5 10 15 17 16 13 18 21 20 19 22 25
 47]
